In [19]:
# Assinatura cega

from Crypto.Hash import SHA
from cryptography.hazmat.primitives.asymmetric import utils

def BlindSigRSA(bits=512):
    """
Função que invoca a geração do par de chaves RSA.
Input: número de bits
Output: chave privada e pública.

Uso: escolhe-se dois números primos p e q, calcula-se a função de Carmichael
φ(n)=(p-1), isto é, d  é o inverso multiplicativo de e mod n, Zn é o grupo,
tal que n é o produto de dois números primos p e q e ambos de grande tamanho.
d- chave privada e (n, e)- chave pública.

Exemplo: Seja 128 o número de bits, suponhamos p=6076863405864212999 e q=1562809530454321421,
então o par de chaves é: chave privada d=4913970632284035618727648810595056111, chave pública:
(n, e)=(9496980045953699178562209452752351579, 404259412098621376441621593718777431).

Atenção: Se os parâmetros p e q forem de tamanhos pequenos, um ataque por força
é trivial. A NIST recomenda no mínimo 3072 bits.
    """
    p, q= next_prime(2**(bits/2)), next_prime(2**(bits))
    n= p*q
    phi=(p-1)*(q-1)
    e=ZZ.random_element(phi)
    while gcd(e, phi) !=1:
        e=ZZ.random_element(phi)
    d= power_mod(e, -1, phi)
    Chave_privada= d
    Chave_publica= (n,e)
    return Chave_privada, Chave_publica

# Ofuscação
def Blind(m, Chave_publica):
    """
Função que invoca a ofuscação da mensagem antes da assinatura.
Input: mensagem e chave pública.
Output: constante de ofuscação e a mensagem ofuscada.

Uso: escolhe-se k pertencente a grupo Zn, e com k e a chave 
pública, o emissor calcula (k**e)*m (ofusca), e envia ao signatário
a mensagem ofuscada m0.

Exemplo: suponhamos que queremos ofuscar m= 70998000111870, então, m0=717349731693766832827349986846885114994760938826138214108
    """
    n, e = Chave_publica
    Zn=IntegerModRing(n)
    k=Zn(randint(2,n-1))
    m0=(k**e)*m
    return (k, m0)

# Momento da assinatura
def Blind_Sign(m0, Chave_privada, Chave_publica):
    n, e = Chave_publica
    d = Chave_privada
    sig = m0**d
    return sig

# Desofuscação
def UnBlind(sig, bfactor, Chave_publica):
    """
Função que invoca a desofuscação do texto ofuscado.
Input: mensagem ofuscada ofuscada, factor de ofuscação e a 
chave pública.
Output: texto claro assinado.

Uso: O emissor multiplica-se o texto ofuscado assinado pelo 
inverso da constante de ofuscação.
    """
    n, _ = Chave_publica
    m1 = sig*(1/bfactor)
    return m1

# converte inteiro para bytes
def int_to_bytes(n):
    """
Função que invoca a conversão de um número inteiro para bytes.
Input: recebe um número inteiro positivo.
Output: retorna um array de bytes.

Uso: a conversão é feita se e só se o número for maior que 0,
se for igual a zero(0), converte para b'\x00'. A
não converte números negativos.

Exemplo: vamos converter para bytes o inteiro 2019.
Em bytes fica: b'\xe3\x07'
    """
    if n > 0:
        arr = []
        while n:
            n, rem = divmod(n, 256)
            arr.append(rem)
        b = bytearray(arr)
    elif n == 0:
        b = bytearray(b'\x00')
    else:
        raise ValueError
    return b

#Hash da mensagem
def hash_sign(m):
    """
Função que invoca o cálculo do Hash da função.
Input: recebe o texto claro.
Outup: retorna o hash da mensagem.

Uso: converte a mensagem para bytes, e em seguida assina o hash.

Exemplo: o hash de m= 2019, seria: '9eaf0bd3a31b2ab38a8b82998d89a9d6b3d9166f'
    """
    h = SHA.new()
    h.update(int_to_bytes(m))
    mhash=h.hexdigest()
    return mhash

# verificação da autenticidade
def verify (m, mhash, Chave_publica, m1):
    """
Função que invoca a verificação da autenticidade.
Input: recebe a mensagem, o hash da mensagem, a chave pública e a mensagem assinada.
Output: retorna dois valores lógicos: True-caso a auteticidade for válida e False-caso
a autenticidade for inválida.

Atenção: caso a autenticidade não for confirmada, o receptor rejeita a mensagem, isto,
a mensagem foi alvo de um ataque.
    """
    n, e = Chave_publica
    h = SHA.new()
    h.update(int_to_bytes(m))
    hash_m=h.hexdigest()
    if hash_m == mhash and m1**e==m:
        return True
    else:
        return False